<center>
<img src="https://raw.githubusercontent.com/afrisenti-semeval/afrisent-semeval-2023/main/afrisenti-logo.png" width="30%" />
</center>

<center>

#SemEval 2023 Shared Task 12: AfriSenti (Task A)

###Starter Notebook

</center>

Baseline code based on the starter code

#1) Installations and imports

##a. Mount drive (if you are running on colab)

##b. Clone or update competition repository

After cloning, under MyDrive, you will see afrisenti-semeval-2023 folder with all the the data for the afrisenti shared task (training and dev) 

In [1]:
import os

# Language to train sentiment classifier for
# am dz ha ig ma pcm pt sw yo

LANGUAGE_CODE = 'yo'
folder = ''

colab = False

if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    proj_folder = '/content/drive/MyDrive'
else:
    proj_folder = os.getcwd()

In [2]:
%cd {proj_folder}


PROJECT_DIR = f'{proj_folder}/afrisent-semeval-2023'
PROJECT_GITHUB_URL = 'https://github.com/afrisenti-semeval/afrisent-semeval-2023.git'

if not os.path.isdir(PROJECT_DIR):
  !git clone {PROJECT_GITHUB_URL}
else:
  %cd {PROJECT_DIR}
  !git pull {PROJECT_GITHUB_URL}

/Users/thomas/afrisent
/Users/thomas/afrisent/afrisent-semeval-2023
From https://github.com/afrisenti-semeval/afrisent-semeval-2023
 * branch            HEAD       -> FETCH_HEAD
Already up to date.


##c. Install required libraries

- Set the project dire
ctory in the cell below, where the requirements file should also be located, and run the cell

In [3]:
%%capture

%cd {PROJECT_DIR}

if os.path.isdir(PROJECT_DIR):
  #The requirements file should be in PROJECT_DIR
  if os.path.isfile(os.path.join(PROJECT_DIR, 'starter_kit/requirements.txt')):
    !pip install -r starter_kit/requirements.txt
  else:
    print('requirements.txt file not found')

else:
  print("Project directory not found, please check again.")

##d. Import libraries

Import libraries below

In [4]:
import pandas as pd
import numpy as np

# Please don not edit anything here
languages = ['am', 'dz', 'ha', 'ig', 'ma', 'pcm', 'pt', 'sw', 'yo']

#2) Dataset

##a. Formatting

The training dataset that was provided for the competition is in the following format:

| ID | text | label |
| --- | --- | --- |
| twt001 | example text | negative |
| twt002 | example text | positive |
| ... | ... | ... |

However, the code in the starter kit do not expect the 
ID and require the training (and evaluation) data to be in the following format

|text | label |
|--- | --- |
|example text | negative |
|example text | positive |
|... | ... |

To reformat the data run the following cell

In [5]:
# Training Data Paths

TASK = 'SubtaskA'
TRAINING_DATA_DIR = os.path.join(PROJECT_DIR, TASK, 'train')
FORMATTED_TRAIN_DATA = os.path.join(TRAINING_DATA_DIR, 'formatted-train-data')

if os.path.isdir(TRAINING_DATA_DIR):
  print('Data directory found.')
  if not os.path.isdir(FORMATTED_TRAIN_DATA):
    print('Creating directory to store formatted data.')
    os.mkdir(FORMATTED_TRAIN_DATA)
else:
  print(TRAINING_DATA_DIR + ' is not a valid directory or does not exist!')

Data directory found.


In [6]:
%cd {TRAINING_DATA_DIR}

training_files = os.listdir()

if len(training_files) > 0:
  for training_file in training_files:
    if training_file.endswith('.tsv'):

      data = training_file.split('_')[0]
      if not os.path.isdir(os.path.join(FORMATTED_TRAIN_DATA, data)):
        print(data, 'Creating directory to store train, dev and test splits.')
        os.mkdir(os.path.join(FORMATTED_TRAIN_DATA, data))
      
      df = pd.read_csv(training_file, sep='\t', names=['ID', 'text', 'label'], header=0)
      df[['text', 'label']].to_csv(os.path.join(FORMATTED_TRAIN_DATA, data, 'train.tsv'), sep='\t', index=False)
    else:
      print(training_file + ' skipped!')
else:
  print('No files are found in this directory!')

/Users/thomas/afrisent/afrisent-semeval-2023/SubtaskA/train
splitted-train-dev-test skipped!
formatted-train-data skipped!
README.txt skipped!


After running the code above, a new folder (called formated-train-data) with formated files is created in the "datasets" folder in the train sub-folder.

##b. <font color='red'>`(Optional) Creating Evaluation (Dev and Test) sets from the available training data`</font>

You may wish to create train and evaluation (dev and test) sets from the training data provided. If you wish to do so, you can run any of the cells below`

###i. If you want to create both the Dev and Test sets, run this cell

In [7]:
if os.path.isdir(FORMATTED_TRAIN_DATA):
  print('Data directory found.')
  SPLITTED_DATA = os.path.join(TRAINING_DATA_DIR, 'splitted-train-dev-test')
  if not os.path.isdir(SPLITTED_DATA):
    print('Creating directory to store train, dev and test splits.')
    os.mkdir(SPLITTED_DATA)
else:
  print(FORMATTED_TRAIN_DATA + ' is not a valid directory or does not exist!')

%cd {FORMATTED_TRAIN_DATA}
formatted_training_files = os.listdir()

if len(formatted_training_files) > 0:
  for data_name in formatted_training_files:
    formatted_training_file = os.path.join(data_name, 'train.tsv')
    if os.path.isfile(formatted_training_file):
      labeled_tweets = pd.read_csv(formatted_training_file, sep='\t', names=['text', 'label'], header=0)
      train, dev, test = np.split(labeled_tweets.sample(frac=1, random_state=42), [int(.7*len(labeled_tweets)), int(.8*len(labeled_tweets))])

      if not os.path.isdir(os.path.join(SPLITTED_DATA, data_name)):
        print(data_name, 'Creating directory to store train, dev and test splits.')
        os.mkdir(os.path.join(SPLITTED_DATA, data_name))

      train.sample(frac=1).to_csv(os.path.join(SPLITTED_DATA, data_name, 'train.tsv'), sep='\t', index=False)
      dev.sample(frac=1).to_csv(os.path.join(SPLITTED_DATA, data_name, 'dev.tsv'), sep='\t', index=False)
      test.sample(frac=1).to_csv(os.path.join(SPLITTED_DATA, data_name,'test.tsv'), sep='\t', index=False)
    else:
      print(training_file + ' is not a supported file!')
else:
  print('No files are found in this directory!')

Data directory found.
/Users/thomas/afrisent/afrisent-semeval-2023/SubtaskA/train/formatted-train-data


After running the code above, a new folder (called splitted-train-dev-test) with train-dev-test split is created in the "datasets" folder in the train sub-folder. Here, the train-dev-test split is 70/10/20



###ii. If you want to create only the Dev set from the training data, please run this

In [8]:
# from sklearn.model_selection import train_test_split

# if os.path.isdir(FORMATTED_TRAIN_DATA):
#   print('Data directory found.')
#   SPLITTED_DATA = os.path.join(TRAINING_DATA_DIR, 'splitted-train-dev')
#   if not os.path.isdir(SPLITTED_DATA):
#     print('Creating directory to store train, dev and test splits.')
#     os.mkdir(SPLITTED_DATA)
# else:
#   print(FORMATTED_TRAIN_DATA + ' is not a valid directory or does not exist!')

# %cd {FORMATTED_TRAIN_DATA}
# formatted_training_files = os.listdir()

# if len(formatted_training_files) > 0:
#   for data_name in formatted_training_files:
#     formatted_training_file = os.path.join(data_name, 'train.tsv')
#     if os.path.isfile(formatted_training_file):
#       labeled_tweets = pd.read_csv(formatted_training_file, sep='\t', names=['text', 'label'], header=0)
#       train, dev = train_test_split(labeled_tweets, test_size=0.3)

#       if not os.path.isdir(os.path.join(SPLITTED_DATA, data_name)):
#         print(data_name, 'Creating directory to store train, dev and test splits.')
#         os.mkdir(os.path.join(SPLITTED_DATA, data_name))

#       train.sample(frac=1).to_csv(os.path.join(SPLITTED_DATA, data_name, 'train.tsv'), sep='\t', index=False)
#       dev.sample(frac=1).to_csv(os.path.join(SPLITTED_DATA, data_name, 'dev.tsv'), sep='\t', index=False)
#     else:
#       print(training_file + ' is not a supported file!')
# else:
#   print('No files are found in this directory!')

After running the code above, a new folder (called splitted-train-dev) with train-dev split is created in the "datasets" folder in the train sub-folder. Here, the train-dev split is 70/30


#3) Training setup

##a. Set project parameters

For a list of models that be used for fine-tuning, you can check [HERE](https://huggingface.co/models).

In [9]:
%cd {PROJECT_DIR}

if LANGUAGE_CODE in languages:
  # Model Training Parameters
  MODEL_NAME_OR_PATH = 'Davlan/afro-xlmr-mini'
  BATCH_SIZE = 32
  LEARNING_RATE = 5e-5
  NUMBER_OF_TRAINING_EPOCHS = 5
  MAXIMUM_SEQUENCE_LENGTH = 128
  SAVE_STEPS = -1

  print('Everything set. You can now start model training.')

else:
  print("Invalid language code/Dataset not released. Please input any of the following released data\n\n\t- 'am'\n\t- 'dz'\n\t- 'ha'\n\t- 'ig'\n\t- 'ma'\n\t- 'pcm'\n\t- 'pt'\n\t- 'sw'\n\t- 'yo'")

/Users/thomas/afrisent/afrisent-semeval-2023
Everything set. You can now start model training.


##b. Train the model

In the section below, we provide three options: 

- 1) training model without any validation; 
- 2) training model with validation but without testing; 
- 3) training a model with validation and test set.

###Model Training



####Starter Code: Datasets, etc


In [10]:
from types import SimpleNamespace
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import warnings
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.optim as optim

import logging
import os
import random
import sys
from dataclasses import dataclass, field
from typing import Optional

import datasets
import numpy as np
import pandas
import pandas as pd
from datasets import load_dataset

import evaluate
import transformers
from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    default_data_collator,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, send_example_telemetry
from transformers.utils.versions import require_version
from datasets import Features, Value, ClassLabel, load_dataset, Dataset

require_version("datasets>=1.8.0", "To fix: pip install -r examples/pytorch/text-classification/requirements.txt")

logger = logging.getLogger(__name__)


with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")


np.random.seed(420)
torch.manual_seed(69)

NOTE: Redirects are currently not supported in Windows or MacOs.


In [11]:
TRAINING_DATA_DIR

'/Users/thomas/afrisent/afrisent-semeval-2023/SubtaskA/train'

In [12]:
DATA_DIR = os.path.join(TRAINING_DATA_DIR, 'splitted-train-dev-test', LANGUAGE_CODE)
EVAL_DIR = os.path.join(PROJECT_DIR, TASK, 'dev')
OUTPUT_DIR = os.path.join(PROJECT_DIR, 'models', LANGUAGE_CODE + '_no_eval')

training_args = TrainingArguments(
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    num_train_epochs=NUMBER_OF_TRAINING_EPOCHS,
    output_dir=OUTPUT_DIR,
    save_steps=SAVE_STEPS,
    overwrite_output_dir=True
    )

In [13]:
data_args = SimpleNamespace(**{
    'max_seq_length': MAXIMUM_SEQUENCE_LENGTH,
    'overwrite_cache': False,
    'pad_to_max_length': True,
    'max_train_samples': None,
    'max_eval_samples': None,
    'max_predict_samples': None
})

model_args = SimpleNamespace(**{
    'model_name_or_path': MODEL_NAME_OR_PATH,
    'config_name': None,
    'tokenizer_name': None,
    'data_dir': DATA_DIR,
    'eval_dir': EVAL_DIR,
    'cache_dir': None,
    'do_lower_case': None,
    'use_fast_tokenizer': True,
    'model_revision': 'main',
    'use_auth_token': False,
    'ignore_mismatched_sizes': False
})


In [14]:
# See all possible arguments in src/transformers/training_args.py
# or by passing the --help flag to this script.
# We now keep distinct sets of args, for a cleaner separation of concerns.

# parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
# model_args, data_args, training_args = parser.parse_args_into_dataclasses()

# Sending telemetry. Tracking the example usage helps us better allocate resources to maintain them. The
# information sent is the one passed as arguments along with your Python/PyTorch versions.
#send_example_telemetry("run_xnli", model_args)

# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    handlers=[logging.StreamHandler(sys.stdout)],
)

log_level = 0 #training_args.get_process_log_level()
logger.setLevel(log_level)
datasets.utils.logging.set_verbosity(log_level)
transformers.utils.logging.set_verbosity(log_level)
transformers.utils.logging.enable_default_handler()
transformers.utils.logging.enable_explicit_format()

In [15]:
# Log on each process the small summary:
logger.warning(
    f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
    + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
)
logger.info(f"Training/evaluation parameters {training_args}")

# Detecting last checkpoint.
last_checkpoint = None
if os.path.isdir(training_args.output_dir) and training_args.do_train and not training_args.overwrite_output_dir:
    last_checkpoint = get_last_checkpoint(training_args.output_dir)
    if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. "
            "Use --overwrite_output_dir to overcome."
        )
    elif last_checkpoint is not None:
        logger.info(
            f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
            "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
        )

10/30/2022 04:27:21 - WARNING - __main__ - Process rank: -1, device: cpu, n_gpu: 0distributed training: False, 16-bits training: False


In [16]:
LANGUAGE_CODE

'yo'

In [17]:
# Set seed before initializing model.
set_seed(training_args.seed)

# In distributed training, the load_dataset function guarantees that only one local process can concurrently
# download the dataset.
# Downloading and loading xnli dataset from the hub.


if training_args.do_train:

    #train_dataset = load_dataset('csv', data_files={'train': model_args.data_dir + '/train.csv'}, cache_dir=model_args.cache_dir)
    #df = train_dataset["train"].to_pandas()
    #label_list = df['label'].unique().tolist()
    #label_list = train_dataset.features["label"].names
    df = pd.read_csv(model_args.data_dir + '/train.tsv', sep='\t')
    df = df.dropna()
    train_dataset = Dataset.from_pandas(df)
    label_list = df['label'].unique().tolist()

if training_args.do_eval:
    #eval_dataset = load_dataset('csv', data_files={'validation': model_args.data_dir + '/dev.csv'}, cache_dir=model_args.cache_dir)

    #df = eval_dataset["validation"].to_pandas()
    #label_list = df['label'].unique().tolist()
    #label_list = eval_dataset.features["label"].names
    df = pd.read_csv(model_args.data_dir + '/dev.tsv', sep='\t')
    df = df.dropna()
    eval_dataset = Dataset.from_pandas(df)
    label_list = df['label'].unique().tolist()

if training_args.do_predict:
    #predict_dataset = load_dataset('csv', data_files={'test': model_args.data_dir + '/test.csv'}, cache_dir=model_args.cache_dir)

    #df = predict_dataset["test"].to_pandas()
    #label_list = df['label'].unique().tolist()
    #label_list = predict_dataset.features["label"].names
    df = pd.read_csv(model_args.data_dir + '/test.tsv', sep='\t')
    df = df.dropna()
    predict_dataset = Dataset.from_pandas(df)
    label_list = df['label'].unique().tolist()

# Labels
num_labels = len(label_list)
print(label_list)

['positive', 'negative', 'neutral']


####Tokenization

In [18]:
# Load pretrained model and tokenizer
# In distributed training, the .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.
#         finetuning_task="xnli",
config = AutoConfig.from_pretrained(
    model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    num_labels=num_labels,
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    do_lower_case=model_args.do_lower_case,
    cache_dir=model_args.cache_dir,
    use_fast=model_args.use_fast_tokenizer,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
)
model = AutoModelForSequenceClassification.from_pretrained(
    model_args.model_name_or_path,
    from_tf=bool(".ckpt" in model_args.model_name_or_path),
    config=config,
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
    ignore_mismatched_sizes=model_args.ignore_mismatched_sizes,
)

[WARNING|modeling_utils.py:2596] 2022-10-30 04:27:24,087 >> Some weights of the model checkpoint at Davlan/afro-xlmr-mini were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
[WARNING|modeling_utils.py:2608] 2022-10-30 04:27:24,088 >> Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at Davlan/afro-xlmr-mini a

In [19]:
# Preprocessing the datasets
# Padding strategy
if data_args.pad_to_max_length:
    padding = "max_length"
else:
    # We will pad later, dynamically at batch creation, to the max sequence length in each batch
    padding = False

# Some models have set the order of the labels to use, so let's make sure we do use it.
label_to_id = None
label_to_id = {v: i for i, v in enumerate(label_list)}

if label_to_id is not None:
    model.config.label2id = label_to_id
    model.config.id2label = {id: label for label, id in config.label2id.items()}

'''
    def preprocess_function(examples):
    # Tokenize the texts
    return tokenizer(
        examples["premise"],
        examples["hypothesis"],
        padding=padding,
        max_length=data_args.max_seq_length,
        truncation=True,
    )
'''

'\n    def preprocess_function(examples):\n    # Tokenize the texts\n    return tokenizer(\n        examples["premise"],\n        examples["hypothesis"],\n        padding=padding,\n        max_length=data_args.max_seq_length,\n        truncation=True,\n    )\n'

In [20]:
def preprocess_function(examples):
    # Tokenize the texts
    #print
    texts =(examples['text'],)
    result = tokenizer(*texts, padding=padding, max_length=data_args.max_seq_length, truncation=True)
    #print(examples['text'])
    #result = tokenizer(examples['text'], examples['text'], padding=padding, max_length=data_args.max_seq_length, truncation=True)
    # Map labels to IDs (not necessary for GLUE tasks)
    if label_to_id is not None and "label" in examples:
        result["label"] = [(label_to_id[l] if l != -1 else -1) for l in examples["label"]]
    
    result['length'], result["tokenized"] = [], []
    for input_ids in result['input_ids']:
        toks = tokenizer.convert_ids_to_tokens(input_ids, skip_special_tokens=True)
        result['length'].append(len(toks)+2)
        result['tokenized'].append(' '.join(toks))
    return result

if training_args.do_train:
    if data_args.max_train_samples is not None:
        max_train_samples = min(len(train_dataset), data_args.max_train_samples)
        train_dataset = train_dataset.select(range(max_train_samples))
    with training_args.main_process_first(desc="train dataset map pre-processing"):
        train_dataset = train_dataset.map(
            preprocess_function,
            batched=True,
            load_from_cache_file=not data_args.overwrite_cache,
            desc="Running tokenizer on train dataset",
        )
    # Log a few random samples from the training set:
    for index in random.sample(range(len(train_dataset)), 3):
        logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

if training_args.do_eval:
    if data_args.max_eval_samples is not None:
        max_eval_samples = min(len(eval_dataset), data_args.max_eval_samples)
        eval_dataset = eval_dataset.select(range(max_eval_samples))
    with training_args.main_process_first(desc="validation dataset map pre-processing"):
        eval_dataset = eval_dataset.map(
            preprocess_function,
            batched=True,
            load_from_cache_file=not data_args.overwrite_cache,
            desc="Running tokenizer on validation dataset",
        )

if training_args.do_predict:
    if data_args.max_predict_samples is not None:
        max_predict_samples = min(len(predict_dataset), data_args.max_predict_samples)
        predict_dataset = predict_dataset.select(range(max_predict_samples))
    with training_args.main_process_first(desc="prediction dataset map pre-processing"):
        predict_dataset = predict_dataset.map(
            preprocess_function,
            batched=True,
            load_from_cache_file=not data_args.overwrite_cache,
            desc="Running tokenizer on prediction dataset",
        )

Running tokenizer on train dataset:   0%|          | 0/6 [00:00<?, ?ba/s]

Running tokenizer on validation dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [21]:
train_dataset, eval_dataset

(Dataset({
     features: ['text', 'label', '__index_level_0__', 'input_ids', 'attention_mask', 'length', 'tokenized'],
     num_rows: 5965
 }),
 Dataset({
     features: ['text', 'label', '__index_level_0__', 'input_ids', 'attention_mask', 'length', 'tokenized'],
     num_rows: 852
 }))

In [22]:
train_text, train_labels = train_dataset['tokenized'], train_dataset['label']

In [23]:
eval_text, eval_labels = eval_dataset['tokenized'], eval_dataset['label']

#### Simple Baselines

Class Proportions (AKA Constant / Majority Classifier Results)

In [24]:
for l in np.unique(eval_labels):
  print(np.mean(np.array(eval_labels) == l), f1_score([l] * len(eval_labels), eval_labels, average='weighted'))

0.38028169014084506 0.5510204081632654
0.2511737089201878 0.401500938086304
0.3685446009389671 0.5385934819897085


In [25]:
# set up unigram, bigram, trigram BOW and TF-IDF

unigram_vectorizer = CountVectorizer(ngram_range=(1, 1), max_features=100000, min_df=7, max_df=0.8)
unigram_vectorizer.fit(train_text)
X_train_unigram = unigram_vectorizer.transform(train_text)
X_test_unigram = unigram_vectorizer.transform(eval_text)

unigram_tf_idf_transformer = TfidfTransformer()
unigram_tf_idf_transformer.fit(X_train_unigram)
X_train_unigram_tf_idf = unigram_tf_idf_transformer.transform(X_train_unigram)
X_test_unigram_tf_idf = unigram_tf_idf_transformer.transform(X_test_unigram)

bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), max_features=100000, min_df=7, max_df=0.8)
bigram_vectorizer.fit(train_text)
X_train_bigram = bigram_vectorizer.transform(train_text)
X_test_bigram = bigram_vectorizer.transform(eval_text)

bigram_tf_idf_transformer = TfidfTransformer()
bigram_tf_idf_transformer.fit(X_train_bigram)
X_train_bigram_tf_idf = bigram_tf_idf_transformer.transform(X_train_bigram)
X_test_bigram_tf_idf = bigram_tf_idf_transformer.transform(X_test_bigram)

trigram_vectorizer = CountVectorizer(ngram_range=(1, 3), max_features=100000, min_df=7, max_df=0.8)
trigram_vectorizer.fit(train_text)
X_train_trigram = trigram_vectorizer.transform(train_text)
X_test_trigram = trigram_vectorizer.transform(eval_text)
trigram_tf_idf_transformer = TfidfTransformer()
trigram_tf_idf_transformer.fit(X_train_trigram)
X_train_trigram_tf_idf =trigram_tf_idf_transformer.transform(X_train_trigram)
X_test_trigram_tf_idf =trigram_tf_idf_transformer.transform(X_test_trigram)



In [26]:
def AdaDTC(**params):
    base_params = {}
    for k, v in list(params.items()):
        if k[:16] == 'base_estimator__':
            base_params[k[16:]] = v
            del params[k]
    return AdaBoostClassifier(base_estimator=DecisionTreeClassifier(**base_params), **params)

classifiers = [
    (MultinomialNB, [
        {'alpha': 1e-3}, {'alpha': 0.1}, {'alpha': 1}, {'alpha': 10}, {'alpha': 1e3},
    ]), 
    (SVC, [
        {'C': 1, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': 1000},
        {'C': 1000.0, 'gamma': 'auto', 'kernel': 'rbf', 'max_iter': 1000}
    ]), 
    (LogisticRegression, [
        {'C': 1, 'max_iter': 1000, 'penalty': 'l2'}
    ]), 
    (RandomForestClassifier, [
        {'criterion': 'gini', 'max_depth': None, 'n_estimators': 500}
    ]), 
    (AdaDTC, [
        {'base_estimator__criterion': 'gini', 'base_estimator__max_depth': 10, 
          'base_estimator__splitter': 'best', 'learning_rate': 0.1, 'n_estimators': 200},
        {'base_estimator__criterion': 'entropy', 'base_estimator__max_depth': 3, 
         'base_estimator__splitter': 'random', 'learning_rate': 1, 'n_estimators': 200}
    ]),  
    (MLPClassifier, [
        {'activation': 'relu', 'alpha': 1, 'hidden_layer_sizes': (100,), 'learning_rate': 'constant'}
    ])
    ]

X_train_data = {'unigram counts': X_train_unigram,
                'unigram tf-idf': X_train_unigram_tf_idf,
                'bigram counts': X_train_bigram, 
                'bigram tf-idf': X_train_bigram_tf_idf,
                'trigram counts': X_train_trigram, 
                'trigram tf-idf': X_train_trigram_tf_idf}

X_test_data = {'unigram counts': X_test_unigram,
                'unigram tf-idf': X_test_unigram_tf_idf,
                'bigram counts': X_test_bigram, 
                'bigram tf-idf': X_test_bigram_tf_idf,
                'trigram counts': X_test_trigram, 
                'trigram tf-idf': X_test_trigram_tf_idf}

def train_and_show_scores(title, model, parameters):

    X_train, X_test = X_train_data[title], X_test_data[title]
    y_train, y_test = train_labels, eval_labels
    
    
    best_train, best_test, best_f1, best_params = 0, 0, 0, None
    for params in parameters:
        clf = model(**params)
        with warnings.catch_warnings(record=True) as w:
            warnings.simplefilter("always")
            clf.fit(X_train, y_train)
        train_score = clf.score(X_train, y_train)
        test_score = clf.score(X_test, y_test)
        test_f1 = f1_score(clf.predict(X_test), y_test, average='weighted')
        
        if test_f1 > best_f1:
            best_train, best_test, best_f1, best_params = train_score, test_score, test_f1, params
    
    print(f"""
    {title}
    Train score: {best_train:.3}
    Eval score: {best_test:.3}
    Weighted Test F1: {best_f1:.3}
    Params: {best_params}
    """)

In [27]:
# Get scores for multiple different models
for model, parameters in classifiers:
    print(model)
    train_and_show_scores('unigram counts', model, parameters)
    train_and_show_scores('unigram tf-idf', model, parameters)
    train_and_show_scores('bigram counts', model, parameters)
    train_and_show_scores('bigram tf-idf', model, parameters)
    train_and_show_scores('trigram counts', model, parameters)
    train_and_show_scores('trigram tf-idf', model, parameters)

<class 'sklearn.naive_bayes.MultinomialNB'>

    unigram counts
    Train score: 0.69
    Eval score: 0.602
    Weighted Test F1: 0.61
    Params: {'alpha': 1}
    

    unigram tf-idf
    Train score: 0.672
    Eval score: 0.588
    Weighted Test F1: 0.629
    Params: {'alpha': 1}
    

    bigram counts
    Train score: 0.765
    Eval score: 0.643
    Weighted Test F1: 0.646
    Params: {'alpha': 0.1}
    

    bigram tf-idf
    Train score: 0.718
    Eval score: 0.616
    Weighted Test F1: 0.655
    Params: {'alpha': 1}
    

    trigram counts
    Train score: 0.74
    Eval score: 0.644
    Weighted Test F1: 0.648
    Params: {'alpha': 1}
    

    trigram tf-idf
    Train score: 0.72
    Eval score: 0.613
    Weighted Test F1: 0.65
    Params: {'alpha': 1}
    
<class 'sklearn.svm._classes.SVC'>

    unigram counts
    Train score: 0.873
    Eval score: 0.617
    Weighted Test F1: 0.63
    Params: {'C': 1, 'gamma': 'scale', 'kernel': 'rbf', 'max_iter': 1000}
    

    unigram tf-i

In [28]:
# Clear Memory
del unigram_vectorizer
del X_train_unigram 
del X_test_unigram

del unigram_tf_idf_transformer
del X_train_unigram_tf_idf
del X_test_unigram_tf_idf
                                                             
del bigram_vectorizer
del X_train_bigram
del X_test_bigram

del bigram_tf_idf_transformer
del X_train_bigram_tf_idf
del X_test_bigram_tf_idf

del trigram_vectorizer
del X_train_trigram
del X_test_trigram

del trigram_tf_idf_transformer
del X_train_trigram_tf_idf
del X_test_trigram_tf_idf

del X_train_data
del X_test_data 

del train_text
del eval_text

In [29]:
1/0

ZeroDivisionError: division by zero